### ingest results file

##### Read the json file using the spark dataframe reader

In [0]:
dbutils.widgets.text("p_data_source", "")
v_data_source = dbutils.widgets.get("p_data_source")

In [0]:
dbutils.widgets.text("p_file_date", "2021-03-21")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, FloatType, DoubleType

In [0]:
results_schema = StructType([
    StructField("resultId", IntegerType(), True),
    StructField("raceId", IntegerType(), True),
    StructField("driverId", IntegerType(), True),
    StructField("constructorId", IntegerType(), True),
    StructField("number", IntegerType(), True),
    StructField("grid", IntegerType(), True),
    StructField("position", IntegerType(), True),
    StructField("positionText", StringType(), True),
    StructField("positionOrder", IntegerType(), True),
    StructField("points", FloatType(), True),
    StructField("laps", IntegerType(), True),
    StructField("time", StringType(), True),
    StructField("milliseconds", IntegerType(), True),
    StructField("fastestLap", IntegerType(), True),
    StructField("rank", IntegerType(), True),
    StructField("fastestLapTime", StringType(), True),
    StructField("fastestLapSpeed", DoubleType(), True),
    StructField("statusId", IntegerType(), True)
])
                                  

In [0]:
results_df = spark.read \
.option("header", True) \
.schema(results_schema) \
.json(f"/mnt/f1datalake2025/raw/{v_file_date}/results.json")


In [0]:
display(results_df)

resultId,raceId,driverId,constructorId,number,grid,position,positionText,positionOrder,points,laps,time,milliseconds,fastestLap,rank,fastestLapTime,fastestLapSpeed,statusId
24986,1053,830,9,33,3,1,1,1,25.0,63,2:02:34.598,7354598,60,2,1:17.524,227.96,1
24987,1053,1,131,44,1,2,2,2,19.0,63,+22.000,7376598,60,1,1:16.702,230.403,1
24988,1053,846,1,4,7,3,3,3,15.0,63,+23.702,7378300,63,3,1:18.259,225.819,1
24989,1053,844,6,16,4,4,4,4,12.0,63,+25.579,7380177,60,6,1:18.379,225.473,1
24990,1053,832,6,55,11,5,5,5,10.0,63,+27.036,7381634,60,7,1:18.490,225.154,1
24991,1053,817,1,3,6,6,6,6,8.0,63,+51.220,7405818,54,12,1:19.341,222.739,1
24992,1053,842,213,10,5,7,7,7,6.0,63,+52.818,7407416,52,9,1:18.994,223.718,1
24993,1053,840,117,18,10,8,8,8,4.0,63,+56.909,7411507,59,8,1:18.782,224.32,1
24994,1053,839,214,31,9,9,9,9,2.0,63,+65.704,7420302,62,15,1:19.422,222.512,1
24995,1053,4,214,14,15,10,10,10,1.0,63,+66.561,7421159,62,14,1:19.417,222.526,1


#####add ingestion date and rename


In [0]:
from pyspark.sql.functions import current_timestamp, lit

In [0]:
results_renamed_df = results_df \
                     .withColumnRenamed("resultId", "result_id") \
                     .withColumnRenamed("raceId", "race_id") \
                     .withColumnRenamed("driverId", "driver_id") \
                     .withColumnRenamed("constructorId", "constructor_id") \
                     .withColumnRenamed("positionText", "position_text") \
                     .withColumnRenamed("positionOrder", "position_order") \
                     .withColumnRenamed("fastestLap", "fastest_lap") \
                     .withColumnRenamed("fastestLapTime", "fastest_lap_time") \
                     .withColumnRenamed("fastestLapSpeed", "fastest_lap_speed") \
                     .withColumnRenamed("statusId", "status_id") \
                     .withColumn("ingestion_date", current_timestamp()) \
                     .withColumn("data_source", lit(v_data_source)) \
                     .withColumn("file_date", lit(v_file_date))
display(results_renamed_df)

result_id,race_id,driver_id,constructor_id,number,grid,position,position_text,position_order,points,laps,time,milliseconds,fastest_lap,rank,fastest_lap_time,fastest_lap_speed,status_id,ingestion_date,data_source,file_date
24986,1053,830,9,33,3,1,1,1,25.0,63,2:02:34.598,7354598,60,2,1:17.524,227.96,1,2025-08-10T06:44:58.28Z,,2021-04-18
24987,1053,1,131,44,1,2,2,2,19.0,63,+22.000,7376598,60,1,1:16.702,230.403,1,2025-08-10T06:44:58.28Z,,2021-04-18
24988,1053,846,1,4,7,3,3,3,15.0,63,+23.702,7378300,63,3,1:18.259,225.819,1,2025-08-10T06:44:58.28Z,,2021-04-18
24989,1053,844,6,16,4,4,4,4,12.0,63,+25.579,7380177,60,6,1:18.379,225.473,1,2025-08-10T06:44:58.28Z,,2021-04-18
24990,1053,832,6,55,11,5,5,5,10.0,63,+27.036,7381634,60,7,1:18.490,225.154,1,2025-08-10T06:44:58.28Z,,2021-04-18
24991,1053,817,1,3,6,6,6,6,8.0,63,+51.220,7405818,54,12,1:19.341,222.739,1,2025-08-10T06:44:58.28Z,,2021-04-18
24992,1053,842,213,10,5,7,7,7,6.0,63,+52.818,7407416,52,9,1:18.994,223.718,1,2025-08-10T06:44:58.28Z,,2021-04-18
24993,1053,840,117,18,10,8,8,8,4.0,63,+56.909,7411507,59,8,1:18.782,224.32,1,2025-08-10T06:44:58.28Z,,2021-04-18
24994,1053,839,214,31,9,9,9,9,2.0,63,+65.704,7420302,62,15,1:19.422,222.512,1,2025-08-10T06:44:58.28Z,,2021-04-18
24995,1053,4,214,14,15,10,10,10,1.0,63,+66.561,7421159,62,14,1:19.417,222.526,1,2025-08-10T06:44:58.28Z,,2021-04-18


In [0]:

results_final_df = results_renamed_df \
                   .drop("status_id")
display(results_final_df)

result_id,race_id,driver_id,constructor_id,number,grid,position,position_text,position_order,points,laps,time,milliseconds,fastest_lap,rank,fastest_lap_time,fastest_lap_speed,ingestion_date,data_source,file_date
24986,1053,830,9,33,3,1,1,1,25.0,63,2:02:34.598,7354598,60,2,1:17.524,227.96,2025-08-10T06:44:58.51Z,,2021-04-18
24987,1053,1,131,44,1,2,2,2,19.0,63,+22.000,7376598,60,1,1:16.702,230.403,2025-08-10T06:44:58.51Z,,2021-04-18
24988,1053,846,1,4,7,3,3,3,15.0,63,+23.702,7378300,63,3,1:18.259,225.819,2025-08-10T06:44:58.51Z,,2021-04-18
24989,1053,844,6,16,4,4,4,4,12.0,63,+25.579,7380177,60,6,1:18.379,225.473,2025-08-10T06:44:58.51Z,,2021-04-18
24990,1053,832,6,55,11,5,5,5,10.0,63,+27.036,7381634,60,7,1:18.490,225.154,2025-08-10T06:44:58.51Z,,2021-04-18
24991,1053,817,1,3,6,6,6,6,8.0,63,+51.220,7405818,54,12,1:19.341,222.739,2025-08-10T06:44:58.51Z,,2021-04-18
24992,1053,842,213,10,5,7,7,7,6.0,63,+52.818,7407416,52,9,1:18.994,223.718,2025-08-10T06:44:58.51Z,,2021-04-18
24993,1053,840,117,18,10,8,8,8,4.0,63,+56.909,7411507,59,8,1:18.782,224.32,2025-08-10T06:44:58.51Z,,2021-04-18
24994,1053,839,214,31,9,9,9,9,2.0,63,+65.704,7420302,62,15,1:19.422,222.512,2025-08-10T06:44:58.51Z,,2021-04-18
24995,1053,4,214,14,15,10,10,10,1.0,63,+66.561,7421159,62,14,1:19.417,222.526,2025-08-10T06:44:58.51Z,,2021-04-18


#####
write the file as parquet to cleaned and processed container

######method 1

In [0]:
#for race_id_list in results_final_df.select("race_id").distinct().collect():
   # if (spark._jsparkSession.catalog().tableExists("f1_processed.results")):
        #spark.sql(f"DELETE FROM f1_processed.results WHERE race_id = {race_id_list.race_id}")

        # very slow but can be improved interms of performance

#####method 2

In [0]:
spark.conf.set("spark.sql.sources.partitionOverwriteMode", "dynamic")

In [0]:
results_final_df = results_final_df.select("result_id", "race_id", "driver_id", "constructor_id", "number", "grid", "position", "position_text", "position_order", "points", "laps", "time", "milliseconds", "fastest_lap", "fastest_lap_time", "fastest_lap_speed", "ingestion_date", "data_source", "file_date")


In [0]:
if (spark._jsparkSession.catalog().tableExists("f1_processed.results")):
    results_final_df.write.mode("overwrite").insertInto("f1_processed.results")
else:
    results_final_df.write.mode("overwrite").format("parquet").saveAsTable("f1_processed.results")

---------------------------------------------------------------------------
NumberFormatException                     Traceback (most recent call last)
File <command-4995747289430837>, line 2
      1 if (spark._jsparkSession.catalog().tableExists("f1_processed.results")):
----> 2     results_final_df.write.mode("overwrite").insertInto("f1_processed.results")
      3 else:
      4     results_final_df.write.mode("overwrite").format("parquet").saveAsTable("f1_processed.results")

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(
     50         module_name, class_name, function_name, time.perf_counter() - start, signature
     51     )
     52     return res

File /databricks/spark/python/pyspark/sql/readwriter.py:1513, in DataFrameWriter.insertInto(self, tableName, overwrite)
   1511 if overwrite is 

In [0]:
dbutils.notebook.exit("Success")

In [0]:
%sql
SELECT raceId, COUNT(1),
FROM f1_processed.results
GROUP BY raceId
ORDER BY raceId DESC

com.databricks.backend.common.rpc.SparkDriverExceptions$SQLExecutionException: org.apache.spark.sql.catalyst.parser.ParseException: 
[PARSE_SYNTAX_ERROR] Syntax error at or near '.'.(line 2, pos 17)

== SQL ==
SELECT raceId, COUNT(1),
FROM f1_processed.results
-----------------^^^
GROUP BY raceId
ORDER BY raceId DESC

	at org.apache.spark.sql.catalyst.parser.ParseException.withCommand(parsers.scala:259)
	at org.apache.spark.sql.catalyst.parser.AbstractParser.parse(parsers.scala:98)
	at org.apache.spark.sql.execution.SparkSqlParser.parse(SparkSqlParser.scala:111)
	at org.apache.spark.sql.catalyst.parser.AbstractSqlParser.parsePlan(AbstractSqlParser.scala:87)
	at com.databricks.sql.parser.DatabricksSqlParser.$anonfun$parsePlan$1(DatabricksSqlParser.scala:77)
	at com.databricks.sql.parser.DatabricksSqlParser.parse(DatabricksSqlParser.scala:98)
	at com.databricks.sql.parser.DatabricksSqlParser.parsePlan(DatabricksSqlParser.scala:74)
	at org.apache.spark.sql.SparkSession.$anonfun$sql$6(SparkSession.scala:919)
	at com.databricks.spark.util.FrameProfiler$.record(FrameProfiler.scala:94)
	at org.apache.spark.sql.catalyst.QueryPlanningTracker.measurePhase(QueryPlanningTracker.scala:383)
	at org.apache.spark.sql.SparkSession.$anonfun$sql$5(SparkSession.scala:918)
	at org.apache.spark.sql.SparkSession.withActive(SparkSession.scala:1148)
	at org.apache.spark.sql.SparkSession.sql(SparkSession.scala:917)
	at org.apache.spark.sql.SparkSession.$anonfun$sql$9(SparkSession.scala:951)
	at org.apache.spark.sql.SparkSession.withActive(SparkSession.scala:1148)
	at org.apache.spark.sql.SparkSession.sql(SparkSession.scala:951)
	at org.apache.spark.sql.SparkSession.sql(SparkSession.scala:984)
	at org.apache.spark.sql.SQLContext.sql(SQLContext.scala:695)
	at com.databricks.backend.daemon.driver.DriverLocal$DbClassicStrategy.executeSQLQuery(DriverLocal.scala:248)
	at com.databricks.backend.daemon.driver.DriverLocal.executeSQLSubCommand(DriverLocal.scala:330)
	at com.databricks.backend.daemon.driver.DriverLocal.$anonfun$executeSql$1(DriverLocal.scala:349)
	at scala.collection.immutable.List.map(List.scala:293)
	at com.databricks.backend.daemon.driver.DriverLocal.executeSql(DriverLocal.scala:344)
	at com.databricks.backend.daemon.driver.JupyterDriverLocal.repl(JupyterDriverLocal.scala:713)
	at com.databricks.backend.daemon.driver.DriverLocal.$anonfun$execute$37(DriverLocal.scala:1043)
	at com.databricks.unity.UCSEphemeralState$Handle.runWith(UCSEphemeralState.scala:45)
	at com.databricks.unity.HandleImpl.runWith(UCSHandle.scala:103)
	at com.databricks.backend.daemon.driver.DriverLocal.$anonfun$execute$26(DriverLocal.scala:1034)
	at com.databricks.logging.UsageLogging.$anonfun$withAttributionContext$1(UsageLogging.scala:426)
	at scala.util.DynamicVariable.withValue(DynamicVariable.scala:62)
	at com.databricks.logging.AttributionContext$.withValue(AttributionContext.scala:196)
	at com.databricks.logging.UsageLogging.withAttributionContext(UsageLogging.scala:424)
	at com.databricks.logging.UsageLogging.withAttributionContext$(UsageLogging.scala:418)
	at com.databricks.backend.daemon.driver.DriverLocal.withAttributionContext(DriverLocal.scala:80)
	at com.databricks.logging.UsageLogging.withAttributionTags(UsageLogging.scala:470)
	at com.databricks.logging.UsageLogging.withAttributionTags$(UsageLogging.scala:455)
	at com.databricks.backend.daemon.driver.DriverLocal.withAttributionTags(DriverLocal.scala:80)
	at com.databricks.backend.daemon.driver.DriverLocal.$anonfun$execute$1(DriverLocal.scala:981)
	at com.databricks.backend.daemon.driver.DriverLocal$.$anonfun$maybeSynchronizeExecution$4(DriverLocal.scala:1422)
	at com.databricks.backend.daemon.driver.DriverLocal.execute(DriverLocal.scala:668)
	at com.databricks.backend.daemon.driver.DriverWrapper.$anonfun$tryExecutingCommand$2(DriverWrapper.scala:859)
	at scala.util.Try$.apply(Try.scala:213)
	at com.databricks.backend.daemon.driver.DriverWrapper.$anonfun$tryExecutingCommand$1(DriverWrapper.scala:851)
	at com.